# **UPerNet(ResNet-50) 기반의 Semantic Segmentation**

## **1. 실습용 리소스 가져오기**

In [ ]:
import gdown
import os

In [ ]:
!mkdir bdd10k
!mkdir pretrained

In [ ]:
dataset_id = '1Dap2l5I9R17fXmZI3HNUFyYos9FgTdvG'
gdown.download(id=dataset_id, output='/content/bdd10k/bdd10k.zip')
os.system("unzip /content/bdd10k/bdd10k.zip")

Downloading...
From (original): https://drive.google.com/uc?id=1Dap2l5I9R17fXmZI3HNUFyYos9FgTdvG
From (redirected): https://drive.google.com/uc?id=1Dap2l5I9R17fXmZI3HNUFyYos9FgTdvG&confirm=t&uuid=f0b0992e-3034-4a77-8ce2-ad7624fb5137
To: /content/bdd10k/bdd10k.zip
100%|██████████| 1.19G/1.19G [00:17<00:00, 68.4MB/s]


0

In [ ]:
!rm ./bdd10k/bdd10k.zip

In [ ]:
pretrained_model_id = '1h1T_JZtauItTeis7MHNCm600idZ2TJYi'
gdown.download(id=pretrained_model_id, output='/content/pretrained/upernet_r50-d8_769x769_40k_sem_seg_bdd100k.pth')

Downloading...
From (original): https://drive.google.com/uc?id=1h1T_JZtauItTeis7MHNCm600idZ2TJYi
From (redirected): https://drive.google.com/uc?id=1h1T_JZtauItTeis7MHNCm600idZ2TJYi&confirm=t&uuid=199dd7f4-1351-4273-af21-bad9283370e3
To: /content/pretrained/upernet_r50-d8_769x769_40k_sem_seg_bdd100k.pth
100%|██████████| 266M/266M [00:03<00:00, 82.9MB/s]


'./pretrained/upernet_r50-d8_769x769_40k_sem_seg_bdd100k.pth'

## **2. PyTorch 기반 구현 실습**

### 2.1 패키지 가져오기

In [ ]:
import os
import random
import json
import time

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights

### 2.2 기반 환경 설정

- 데이터셋 및 사전학습모델 경로 설정

In [ ]:
BDD10K_DATA_ROOT_PATH = "/content/bdd10k"

In [ ]:
PRETRAINED_MODEL_PATH = "/content/pretrained/upernet_r50-d8_769x769_40k_sem_seg_bdd100k.pth"

- BDD10K Segmentation 클래스 정의
    - BDD10K 데이터셋의 원본 Semantic Segmentation 라벨 구조
        - 주행 가능 영역과 차선에 대해 상당히 세분화된 픽셀 라벨을 제공
            - drivable_maps (주행 가능 영역 마스크):
                - 픽셀 값 0: background (배경, 즉 주행 불가능한 영역)
                - 픽셀 값 1: direct (현재 차량이 직접 주행할 수 있는 영역)
                - 픽셀 값 2: alternative (차량은 주행할 수 있지만, 현재 차선이 아닌 다른 주행 가능 영역)
                - 목적: 차량이 안전하게 이동할 수 있는 공간 파악
            - lane_masks (차선 마스크):
                - 픽셀 값 0: background (배경)
                - 픽셀 값 1: road_line (도로의 기본적인 차선 경계선)
                - 픽셀 값 2: other_line (횡단보도 선, 정지선 등 기타 도로 위의 선)
                - 목적: 차량의 횡방향 위치 제어(차선 유지)와 차선 변경 판단에 사용
    - 실습에 적용된 구조
        -  실습 목적: **차선 및 주행 가능 영역 인식을 이해**하는 것
        - background: 0: 주행과 무관한 모든 픽셀 (가장 낮은 ID)
        - drivable: 1: 차량이 안전하게 주행할 수 있는 모든 영역 (원본의 direct와 alternative를 모두 포함)
        - lane: 2: 도로 위의 모든 차선 (원본의 road_line과 other_line을 모두 포함)

In [ ]:
CLASS_LABELS = {
    "background": 0,
    "drivable": 1,
    "lane": 2,
    # "road_line": 2,
    # "other_line": 3
}

- 분류 클래스의 개수 = 배경 + 주행 가능 영역 + 차선 = 3

In [ ]:
NUM_CLASSES = len(CLASS_LABELS)

- UPerNet 모델의 입력 이미지 크기 (사전 학습 모델과 동일하게 맞춤)

In [ ]:
INPUT_IMAGE_HEIGHT, INPUT_IMAGE_WIDTH = 769, 769

- GPU 설정

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


### 2.3 구성요소 구현

#### 2.3.1 BDD10K Semantic Segmentation 데이터셋 로더

- BDD10K 데이터셋의 Semantic Segmentation 라벨 구조에 맞춰 수정
- JSON에 "segmentation" 필드가 직접 마스크 경로를 제공하지 않으므로, 규칙 기반으로 'drivable_maps'와 'lane_masks' 폴더에서 마스크를 찾아야 함
- 예시
    - 원본 이미지: 'val/b1c4c1a2-3f2d0111.jpg'
    - drivable 마스크: 'labels/drivable_maps/10k/val/b1c4c1a2-3f2d0111.png'
    - lane 마스크: 'labels/lane_masks/10k/val/b1c4c1a2-3f2d0111.png'

In [ ]:
def get_bdd10k_segmentation_paths(split='val'):
    if split == 'train':
        image_dir = os.path.join(BDD10K_DATA_ROOT_PATH, 'train')
        drivable_mask_dir = os.path.join(BDD10K_DATA_ROOT_PATH, 'labels', 'drivable_maps', '10k', 'train')
        lane_mask_dir = os.path.join(BDD10K_DATA_ROOT_PATH, 'labels', 'lane_masks', '10k', 'train')

    elif split == 'val':
        image_dir = os.path.join(BDD10K_DATA_ROOT_PATH, 'val')
        drivable_mask_dir = os.path.join(BDD10K_DATA_ROOT_PATH, 'labels', 'drivable_maps', '10k', 'val')
        lane_mask_dir = os.path.join(BDD10K_DATA_ROOT_PATH, 'labels', 'lane_masks', '10k', 'val')

    else:
        raise ValueError(f"지원하지 않는 split: {split}. 'train', 'val' 중 하나여야 합니다.")

    samples = []

    # 이미지 파일들을 기준으로 마스크를 찾음
    for image_name in os.listdir(image_dir):
        if image_name.endswith('.jpg'):
            base_name = os.path.splitext(image_name)[0]

            image_path = os.path.join(image_dir, image_name)
            drivable_mask_path = os.path.join(drivable_mask_dir, base_name + '.png')
            lane_mask_path = os.path.join(lane_mask_dir, base_name + '.png')

            # 모든 파일이 존재하는지 확인 (필수)
            if os.path.exists(image_path) and os.path.exists(drivable_mask_path) and os.path.exists(lane_mask_path):
                samples.append({
                    'image_path': image_path,
                    'drivable_mask_path': drivable_mask_path,
                    'lane_mask_path': lane_mask_path,
                    'image_name': image_name
                })

            # else:
            #     print(f"누락된 파일: {image_name} 관련 마스크 파일이 없습니다.")

    # 안정적인 학습/추론을 위해 리스트를 정렬
    samples = sorted(samples, key=lambda x: x['image_name'])
    return samples

- 학습, 검증을 하지 않고 사전학습된 모델을 로드하여 이용할 때에는 단순히 파일만 가져오도록 함
    - 'val' 디렉토리의 JPG 파일 목록만 가져옴

In [ ]:
def get_image_paths_from_dir(target_dir):
    """
    지정된 디렉토리에서 모든 JPG 이미지 파일의 경로를 가져옵니다.
    """
    if not os.path.exists(target_dir):
        print(f"CRITICAL ERROR: Directory does not exist: {target_dir}")
        return []

    image_paths = []
    for fname in os.listdir(target_dir):
        if fname.lower().endswith(('.jpg', '.jpeg')):
            image_paths.append(os.path.join(target_dir, fname))

    if not image_paths:
        print(f"WARNING: No JPG/JPEG images found in {target_dir}. Check directory content or file extensions.")
        return []

    # 랜덤 선택을 위해 이미지 이름을 기준으로 정렬은 불필요하지만 일관성을 위해 유지
    return sorted(image_paths)

- BDD10KSegmentationDataset 클래스
    - 마스크를 합쳐 최종 Segmentation Target 생성
        - BDD10K의 기준
            - drivable_maps
                - 0: background, 1: direct drivable, 2: alternative drivable
            - lane_masks
                - 0: background, 1: road line, 2: other lane line
        - 실습에서의 기준(단순화)
            - 0: background, 1: drivable area, 2: lane_lines

In [ ]:
# BDD10KSegmentationDataset 클래스는 학습시에만 사용되므로 주석 처리하거나 제거 가능
class BDD10KSegmentationDataset(torch.utils.data.Dataset):
    def __init__(self, split='val', transform=None, target_transform=None):
        self.samples = get_bdd10k_segmentation_paths(split)
        self.transform = transform
        self.target_transform = target_transform # 마스크 변환용

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample_info = self.samples[idx]

        # 이미지 로드 (RGB)
        image = Image.open(sample_info['image_path']).convert('RGB')

        # 주행 가능 영역 마스크 로드 (Grayscale)
        drivable_mask = Image.open(sample_info['drivable_mask_path']).convert('L')
        # 차선 마스크 로드 (Grayscale)
        lane_mask = Image.open(sample_info['lane_mask_path']).convert('L')

        # 이미지 크기와 동일한 최종 마스크 생성 (all zeros initially for background)
        final_mask_np = np.zeros(image.size[::-1], dtype=np.uint8) # (H, W)

        drivable_mask_np = np.array(drivable_mask)
        lane_mask_np = np.array(lane_mask)

        # 1. 주행 가능 영역 (ID 1) 설정
        # drivable_mask_np의 픽셀 값이 1 (direct) 또는 2 (alternative)인 부분을 ID 1로 설정
        final_mask_np[np.where(drivable_mask_np > 0)] = CLASS_LABELS["drivable"] # drivable_mask_np > 0

        # 2. 차선 (ID 2) 설정 - 차선은 주행 가능 영역 위에 덮어씌움 (더 중요한 요소)
        # lane_mask_np의 픽셀 값이 1 (road line) 또는 2 (other lane line)인 부분을 ID 2로 설정
        final_mask_np[np.where(lane_mask_np > 0)] = CLASS_LABELS["lane"] # lane_mask_np > 0

        target_mask = Image.fromarray(final_mask_np)

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            target_mask = self.target_transform(target_mask)
        else: # target_transform이 없으면 기본적으로 텐서로 변환
            target_mask = torch.from_numpy(np.array(target_mask, dtype=np.int64))

        return image, target_mask, sample_info['image_name'] # image_name도 반환하여 추론 결과에 사용

#### 2.3.2 Minimal UPerNet (ResNet50 백본) 구현

- torchvision의 ResNet50을 사용하고, FPN과 PPM을 간략화하여 구현
    - FPN (Feature Pyramid Network)
    - PPM (Pyramid Pooling Module)


In [ ]:
class PPM(nn.Module):
    def __init__(self, in_dim, reduction_dim, bins):
        super(PPM, self).__init__()
        self.features = []
        for bin in bins:
            self.features.append(nn.Sequential(
                nn.AdaptiveAvgPool2d(bin),
                nn.Conv2d(in_dim, reduction_dim, kernel_size=1, bias=False),
                nn.BatchNorm2d(reduction_dim),
                nn.ReLU(inplace=True)
            ))
        self.features = nn.ModuleList(self.features)

    def forward(self, x):
        x_size = x.size()
        out = [x]
        for f in self.features:
            out.append(F.interpolate(f(x), x_size[2:], mode='bilinear', align_corners=True))
        return torch.cat(out, 1)

In [ ]:
class UPerNet(nn.Module):
    def __init__(self, num_classes, backbone_name='resnet50', pretrained_backbone=True):
        super(UPerNet, self).__init__()

        # 백본 (Feature Extractor) - ResNet50 사용
        if backbone_name == 'resnet50':
            # weights=ResNet50_Weights.IMAGENET1K_V1: ImageNet으로 사전 학습된 가중치 로드
            resnet = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1 if pretrained_backbone else None)

            # ResNet의 각 Stage에서 특징맵을 추출
            # C1 = conv1, bn1, relu, maxpool
            # C2 = layer1
            # C3 = layer2
            # C4 = layer3
            # C5 = layer4

            # 실제 FPN은 C2, C3, C4, C5를 사용함
            # `self.resnet_features`는 FPN에 직접 전달될 특징 맵들을 저장함
            self.backbone = nn.Sequential(
                resnet.conv1, resnet.bn1, resnet.relu, resnet.maxpool, # Initial layers (before C2)
                resnet.layer1, # C2 output: 256
                resnet.layer2, # C3 output: 512
                resnet.layer3, # C4 output: 1024
                resnet.layer4  # C5 output: 2048
            )

            # ResNet Stage별 출력 채널
            # ResNet50: C2=256, C3=512, C4=1024, C5=2048
            self.in_channels = [256, 512, 1024, 2048]
        else:
            raise NotImplementedError(f"Backbone {backbone_name} not supported yet.")

        # --- FPN & PPM 관련 채널 설정 재확인 ---
        # UPerNet은 PPM의 출력을 가장 높은 피라미드 레벨의 FPN에 통합합니다.
        # 즉, C5 특징맵을 PPM에 넣고, 그 결과를 FPN의 시작점으로 사용합니다.

        self.ppm_out_channels = 512 # PPM의 최종 출력 채널
        self.ppm = PPM(self.in_channels[-1], self.ppm_out_channels // 4, bins=(1, 2, 3, 6))
        self.ppm_conv = nn.Sequential(
            nn.Conv2d(self.in_channels[-1] + self.ppm_out_channels, self.ppm_out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(self.ppm_out_channels),
            nn.ReLU(inplace=True),
            nn.Dropout(0.1)
        )

        # FPN 입력 채널들: C2, C3, C4
        # self.in_channels[:-1]은 [256, 512, 1024]
        self.fpn_in_channels = self.in_channels[:-1] # ResNet C2, C3, C4
        self.fpn_out_channels = 256 # FPN 각 단계의 출력 채널 (일반적으로 256)

        self.fpn_convs = nn.ModuleList() # 1x1 conv for FPN lateral connections
        self.fpn_post_convs = nn.ModuleList() # 3x3 conv for FPN output features

        for in_dim in reversed(self.fpn_in_channels): # C4(1024) -> C3(512) -> C2(256) 순서로 처리
            self.fpn_convs.append(nn.Conv2d(in_dim, self.fpn_out_channels, kernel_size=1, bias=False))
            self.fpn_post_convs.append(nn.Sequential(
                nn.Conv2d(self.fpn_out_channels, self.fpn_out_channels, kernel_size=3, padding=1, bias=False),
                nn.BatchNorm2d(self.fpn_out_channels),
                nn.ReLU(inplace=True)
            ))

        # 최종 분류기 (Classifier)
        # PPM 출력 채널 + 모든 FPN 출력 채널을 합친 후 num_classes 채널로
        self.final_head = nn.Sequential(
            nn.Conv2d(self.ppm_out_channels + len(self.fpn_in_channels) * self.fpn_out_channels,
                      num_classes, kernel_size=1)
        )

    def _forward_backbone(self, x):
        # ResNet의 각 Stage에서 특징맵을 추출
        # C1 = conv1, bn1, relu, maxpool
        # C2 = layer1
        # C3 = layer2
        # C4 = layer3
        # C5 = layer4

        # torchvision resnet의 경우, layer1, layer2, layer3, layer4가 각각 C2, C3, C4, C5에 해당
        x = self.backbone[0](x) # conv1, bn1, relu, maxpool
        c2 = self.backbone[1](x) # layer1
        c3 = self.backbone[2](c2) # layer2
        c4 = self.backbone[3](c3) # layer3
        c5 = self.backbone[4](c4) # layer4

        return [c2, c3, c4, c5] # List of feature maps from C2 to C5

    def forward(self, x):
        input_size = x.size()[2:] # (H, W)

        # 백본을 통해 특징맵 추출
        c_features = self._forward_backbone(x) # [c2, c3, c4, c5]

        # C5 특징맵에 PPM 적용
        ppm_out = self.ppm(c_features[-1]) # c_features[-1]은 c5
        ppm_out = self.ppm_conv(ppm_out) # (B, ppm_out_channels, H_c5, W_c5)

        # FPN (Feature Pyramid Network)
        # top-down path
        fpn_out_list = [ppm_out] # PPM 출력이 FPN의 가장 높은 레벨 출력으로 시작

        # ResNet 특징 맵은 [C2, C3, C4, C5] 순서
        # FPN은 C4 -> C3 -> C2 역순으로 합쳐나감.
        # c_features[:-1]은 [c2, c3, c4]
        # reversed(self.fpn_in_channels)는 [1024, 512, 256]

        current_fpn_feature = ppm_out # P5 (C5)에서 시작하는 FPN 특징

        # Zip fpn_convs with reversed(c_features[:-1])
        # self.fpn_convs는 (C4->256), (C3->256), (C2->256)
        # c_features[:-1]은 [C2, C3, C4]

        # 루프를 돌면서 C4, C3, C2에 해당하는 특징맵을 사용해야 합니다.
        # c_features[-2]는 C4, c_features[-3]은 C3, c_features[-4]는 C2

        for i, lateral_conv in enumerate(self.fpn_convs):
            # 이전 FPN 레벨의 특징맵을 현재 스케일로 upsample
            # 현재 current_fpn_feature의 스케일 (H, W)
            target_size = c_features[-(i+2)].size()[2:] # 예를 들어, i=0일 때 c_features[-2]는 C4
            upsampled_current_fpn_feature = F.interpolate(current_fpn_feature, size=target_size, mode='bilinear', align_corners=True)

            # 현재 ResNet 특징맵 (C4, C3, C2)을 1x1 컨볼루션으로 채널 맞춤 (lateral connection)
            # 여기였던 self.fpn_in_convs[i](c[i+1])가 문제였는데, c_features[-(i+2)]로 직접 참조합니다.
            # self.fpn_convs[i]는 lateral_conv에 해당
            lateral_feature = lateral_conv(c_features[-(i+2)]) # c_features[-2]는 C4, c_features[-3]는 C3, c_features[-4]는 C2

            # FPN Add
            current_fpn_feature = lateral_feature + upsampled_current_fpn_feature

            # 3x3 conv (post-fusion)
            current_fpn_feature = self.fpn_post_convs[i](current_fpn_feature)

            fpn_out_list.append(current_fpn_feature)

        # UPerNet은 모든 FPN 레벨의 출력을 원래 입력 크기로 upsample한 후 concatenate
        # fpn_out_list에는 [PPM_out(P5), P4, P3, P2] 순서로 담겨있습니다.

        all_upsampled_fpn_features = []
        for feature in fpn_out_list:
            all_upsampled_fpn_features.append(F.interpolate(feature, size=input_size, mode='bilinear', align_corners=True))

        # 모든 업샘플링된 특징들을 채널 방향으로 합칩니다.
        concat_features = torch.cat(all_upsampled_fpn_features, dim=1) # (B, total_channels, H, W)

        # 최종 분류기 헤드
        output = self.final_head(concat_features)

        return output

#### 2.3.3 데이터 전처리 및 후처리 변환

In [ ]:
# ImageNet으로 사전 학습된 ResNet50 백본을 위한 정규화 값 사용
TRANSFORM_IMG = transforms.Compose([
    transforms.Resize((INPUT_IMAGE_HEIGHT, INPUT_IMAGE_WIDTH)),
    transforms.ToTensor(), # (H, W, C) -> (C, H, W), 0-255 -> 0-1
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 마스크는 Tensor로 변환하고, 픽셀 값 그대로 사용 (클래스 ID)
TRANSFORM_MASK = transforms.Compose([
    transforms.Resize((INPUT_IMAGE_HEIGHT, INPUT_IMAGE_WIDTH), interpolation=Image.NEAREST), # 마스크는 Nearest Neighbor 보간
    transforms.ToTensor(), # 0-255 -> 0-1.0
    # 마스크는 클래스 ID이므로 정규화하지 않음. ToTensor() 이후 (1, H, W) 형태로 float32
    # 나중에 .long()로 int64로 변환하여 Loss 함수에 전달해야 함.
])

#### 2.3.4 결과 시각화 유틸리티 함수

In [ ]:
COLOR_MAP = {
    0: (0, 0, 0),       # Background (Black)
    1: (0, 255, 0),     # Drivable Area (Green)
    2: (255, 0, 0),     # Lane Lines (Red)
    # 3: (0, 0, 255)    # Other lines (Blue) (사용 안 함)
}

In [ ]:
def decode_segmentation_map(mask):
    """
    예측된 클래스 ID 마스크 (numpy array)를 컬러 이미지로 변환
    """
    height, width = mask.shape
    color_mask = np.zeros((height, width, 3), dtype=np.uint8)
    for class_id, color in COLOR_MAP.items():
        color_mask[mask == class_id] = color
    return color_mask

### 2.4 학습 부분 (코드로만 제시, 실행하지 않음)

In [ ]:
def train_upernet_model(model, train_loader, val_loader, num_epochs=10, learning_rate=0.001):
    model.train() # 모델을 훈련 모드로 전환
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss(ignore_index=CLASS_LABELS["background"]) # 배경 픽셀은 Loss 계산에서 제외

    print("\n--- UPerNet 모델 학습 시작 (실제로 실행되지 않음) ---")
    print("BDD10K 데이터셋으로 UPerNet을 학습시키려면 상당한 시간과 GPU 자원이 필요합니다.")
    print("이 코드는 개념 이해를 위한 것이며, 실제 학습은 다음처럼 진행될 것입니다:\n")

    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (images, masks, _) in enumerate(train_loader):
            images = images.to(device)
            masks = masks.squeeze(1).long().to(device) # (B, 1, H, W) -> (B, H, W) for CrossEntropyLoss

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if (i + 1) % 100 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss / (i+1):.4f}")

        # 검증 루프 (생략)
        # model.eval()
        # with torch.no_grad(): ...

        print(f"Epoch [{epoch+1}/{num_epochs}] 평균 Loss: {running_loss / len(train_loader):.4f}")
        # 모델 저장 (예: torch.save(model.state_dict(), f"upernet_epoch_{epoch+1}.pth"))

    print("\n--- UPerNet 모델 학습 완료 (가정) ---")


### 2.5 사전 학습 모델 로드 및 추론 (핵심 실행 부분)

- BDD100K 데이터셋을 대상으로 Validation까지 수행할 때 사용

In [ ]:
def run_segmentation_inference(model, val_dataset, num_samples_to_show=5):
    model.eval() # 모델을 평가 모드로 전환

    print(f"\n--- 사전 학습 모델 로드 및 추론 시작 ---")
    if not os.path.exists(PRETRAINED_MODEL_PATH):
        print(f"오류: 사전 학습 모델 파일이 없습니다: {PRETRAINED_MODEL_PATH}")
        print("파일 경로를 확인하거나, 해당 파일을 다운로드하여 스크립트와 같은 위치에 배치하세요.")
        return

    try:
        model.load_state_dict(torch.load(PRETRAINED_MODEL_PATH, map_location=device), strict=False) # strict=False는 일부 레이어가 없을 때 유연하게 처리
        print(f"'{PRETRAINED_MODEL_PATH}' 모델 가중치를 성공적으로 로드했습니다.")
    except Exception as e:
        print(f"모델 가중치 로드 중 오류 발생: {e}")
        print("모델 아키텍처와 .pth 파일의 가중치가 호환되는지 확인하세요.")
        return

    plt.figure(figsize=(20, num_samples_to_show * 5))

    # DataLoader를 사용하여 배치 단위로 이미지 가져오기
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False)

    sample_count = 0
    with torch.no_grad(): # 추론 시에는 그라디언트 계산 비활성화
        for i, (images, masks_gt, image_name) in enumerate(val_loader):
            if sample_count >= num_samples_to_show:
                break

            images = images.to(device)
            # masks_gt = masks_gt.squeeze(1).long().to(device) # Ground Truth 마스크 (옵션)

            # 추론 실행
            outputs = model(images) # (B, num_classes, H, W)

            # 예측된 마스크 (가장 높은 확률을 가진 클래스 ID)
            predicted_mask = torch.argmax(outputs, dim=1).squeeze(0).cpu().numpy() # (H, W)

            # 원본 이미지 (PyTorch Tensor -> NumPy RGB)
            original_image_np = images.squeeze(0).cpu().numpy()
            original_image_np = np.transpose(original_image_np, (1, 2, 0)) # (C, H, W) -> (H, W, C)
            original_image_np = (original_image_np * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]) * 255
            original_image_np = original_image_np.astype(np.uint8)

            # 예측된 마스크를 컬러로 디코딩
            predicted_color_mask = decode_segmentation_map(predicted_mask)

            # 원본 이미지와 예측된 마스크를 합성하여 시각화
            # 알파 블렌딩 (원본 이미지는 RGB, 마스크는 컬러, 두 개를 투명하게 합성)
            blended_image = cv2.addWeighted(original_image_np, 0.7, predicted_color_mask, 0.3, 0)

            plt.subplot(num_samples_to_show, 1, sample_count + 1)
            plt.imshow(blended_image)
            plt.title(f"Segmentation Result for {image_name[0]}")
            plt.axis('off')

            sample_count += 1

    plt.tight_layout()
    plt.show()
    print("--- 추론 및 시각화 완료 ---")

- BDD10K 등을 대상으로 실제 예측에만 사용하기위하여 샘플링한 데이터에만 추론 예측을 적용함

In [ ]:
def run_segmentation_inference_on_random_samples(model, image_paths_list, preprocess_transform, num_samples_to_show=5):
    model.eval() # 모델을 평가 모드로 전환

    print(f"\n--- 사전 학습 모델 로드 및 랜덤 {num_samples_to_show}개 샘플 추론 시작 ---")
    if not os.path.exists(PRETRAINED_MODEL_PATH):
        print(f"오류: 사전 학습 모델 파일이 없습니다: {PRETRAINED_MODEL_PATH}")
        print("파일 경로를 확인하거나, 해당 파일을 다운로드하여 스크립트와 같은 위치에 배치하세요.")
        return

    try:
        model.load_state_dict(torch.load(PRETRAINED_MODEL_PATH, map_location=device), strict=False)
        print(f"'{PRETRAINED_MODEL_PATH}' 모델 가중치를 성공적으로 로드했습니다.")
    except Exception as e:
        print(f"모델 가중치 로드 중 오류 발생: {e}")
        print("모델 아키텍처와 .pth 파일의 가중치가 호환되는지 확인하세요.")
        return

    if len(image_paths_list) < num_samples_to_show:
        print(f"경고: 사용 가능한 이미지({len(image_paths_list)}개)가 요청한 수({num_samples_to_show}개)보다 적습니다. 사용 가능한 모든 이미지를 추론합니다.")
        samples_to_infer_paths = image_paths_list
    else:
        samples_to_infer_paths = random.sample(image_paths_list, num_samples_to_show) # 랜덤으로 샘플 선택

    plt.figure(figsize=(20, num_samples_to_show * 5))

    with torch.no_grad():
        for i, image_path in enumerate(samples_to_infer_paths):
            image_name = os.path.basename(image_path)

            # 이미지 로드 (RGB)
            original_image = Image.open(image_path).convert('RGB')

            # 전처리
            input_tensor = preprocess_transform(original_image).unsqueeze(0).to(device) # (1, C, H, W)

            # 추론 실행
            start_time = time.time()
            outputs = model(input_tensor) # (B, num_classes, H, W)
            inference_time = time.time() - start_time

            # 예측된 마스크 (가장 높은 확률을 가진 클래스 ID)
            predicted_mask = torch.argmax(outputs, dim=1).squeeze(0).cpu().numpy() # (H, W)

            # 원본 이미지 (PIL Image -> NumPy RGB, 시각화를 위해 원본 크기 유지)
            original_image_np = np.array(original_image)

            # 예측된 마스크를 원본 이미지 크기로 리사이즈 후 컬러 디코딩
            # 모델 출력(predicted_mask)은 769x769, 원본 이미지는 1280x720
            # 마스크를 원본 크기로 리사이즈해야 정확히 오버레이 가능
            predicted_mask_resized = cv2.resize(predicted_mask.astype(np.uint8),
                                                (original_image_np.shape[1], original_image_np.shape[0]),
                                                interpolation=cv2.INTER_NEAREST)

            predicted_color_mask = decode_segmentation_map(predicted_mask_resized)

            # 원본 이미지와 예측된 마스크를 합성하여 시각화
            blended_image = cv2.addWeighted(original_image_np, 0.7, predicted_color_mask, 0.3, 0) # 0.3은 마스크 투명도

            plt.subplot(num_samples_to_show, 1, i + 1)
            plt.imshow(blended_image)
            plt.title(f"[{image_name}] Inference Time: {inference_time:.3f}s")
            plt.axis('off')

    plt.tight_layout()
    plt.show()
    print("--- 추론 및 시각화 완료 ---")

### 2.6 메인 실행 블록

In [ ]:
if __name__ == '__main__':
    # 1. BDD10K 데이터셋 로드
    val_dataset = BDD10KSegmentationDataset(split='val', transform=TRANSFORM_IMG, target_transform=TRANSFORM_MASK)
    print(f"BDD10K Validation 데이터셋 로드 완료: {len(val_dataset)}개 이미지")

    if not val_dataset:
        print("로드된 BDD10K Segmentation 데이터가 없습니다. 경로 설정을 확인해주세요.")
    else:
        # 2. UPerNet 모델 생성
        # ResNet50 백본을 사용하고, num_classes는 위에서 정의한 3개 클래스 (배경, 주행 가능, 차선)
        model = UPerNet(num_classes=NUM_CLASSES, backbone_name='resnet50', pretrained_backbone=False).to(device)
        print("UPerNet 모델 생성 완료 (ResNet50 백본).")

        # 3. 학습 부분 (실행하지 않음)
        # train_upernet_model(model, train_loader, val_loader)

        # 4. 사전 학습 모델 로드 및 추론 실행
        run_segmentation_inference(model, val_dataset, num_samples_to_show=5)

BDD10K Validation 데이터셋 로드 완료: 0개 이미지
로드된 BDD10K Segmentation 데이터가 없습니다. 경로 설정을 확인해주세요.


In [ ]:
if __name__ == '__main__':
    # 'val' 디렉토리의 절대 경로 설정
    val_image_directory = os.path.join(BDD10K_DATA_ROOT_PATH, "val")

    # 'val' 디렉토리에서 JPG 이미지 파일 경로 목록만 가져옴
    image_paths_for_inference = get_image_paths_from_dir(val_image_directory)
    print(f"BDD10K Validation 이미지 디렉토리에서 {len(image_paths_for_inference)}개 이미지 경로 로드 완료.")

    if not image_paths_for_inference:
        print("로드된 BDD10K 이미지 경로가 없습니다. 'val' 디렉토리 또는 경로 설정을 확인해주세요.")

    else:
        # UPerNet 모델 생성
        model = UPerNet(num_classes=NUM_CLASSES, backbone_name='resnet50', pretrained_backbone=False).to(device)
        print("UPerNet 모델 생성 완료 (ResNet50 백본).")

        run_segmentation_inference_on_random_samples(model, image_paths_for_inference, TRANSFORM_IMG, num_samples_to_show=5)

BDD10K Validation 이미지 디렉토리에서 1000개 이미지 경로 로드 완료.
UPerNet 모델 생성 완료 (ResNet50 백본).

--- 사전 학습 모델 로드 및 랜덤 5개 샘플 추론 시작 ---
'/content/pretrained/upernet_r50-d8_769x769_40k_sem_seg_bdd100k.pth' 모델 가중치를 성공적으로 로드했습니다.


RuntimeError: Given groups=1, weight of size [128, 2048, 1, 1], expected input[1, 256, 1, 1] to have 2048 channels, but got 256 channels instead

<Figure size 2000x2500 with 0 Axes>